In [1]:
import os
import json
import csv
import numpy as np
import pandas as pd
import re

In [83]:

path = '' 
## path to tsv file with MQM annotations

#df = pd.read_csv(path, sep='\t')
data=[]

with open(path) as f:
    for line in f.readlines():
        fields = line.strip().split('\t')
        if len(fields)>10:
            print('erroneous line:'+line)
            break 
        else:
            if len(fields)==9:
                fields.append('')
            data.append(fields)
    col = data[0]
    # read into df for visualisation purposes
    df =pd.DataFrame(data[1:], columns=col)
 
df.columns


Index(['system', 'doc', 'doc_id', 'seg_id', 'rater', 'source', 'target',
       'category', 'severity', ''],
      dtype='object')

In [84]:
import re 

def find_indices(astring, apattern):
    match=(re.search(apattern, astring))

    if match != None:
        return match.span()
    else:
        return None



In [86]:

   
pre_sid = ''
indices = {}
sources = {}
targets = {}
severities = {}

for i,r in df.iterrows():
    seg_id = str(r.seg_id).strip()
    doc_id = str(r.doc_id).strip()
    system = r.system.strip()
    doc = r.doc.strip()
    rater = r.rater.strip()
    source = r.source.strip()
    target = r.target.strip()
    severity = r.severity.strip().lower()
    main_source = source.replace("<v>","").replace("</v>","")
    main_source = re.sub(' +', ' ', main_source)
    main_target = target.replace("<v>","").replace("</v>","")
    main_target = re.sub(' +', ' ', main_target)
    
    sid = system + '_' + doc + '_' + doc_id + '_' + rater
    if pre_sid!=sid :
        pre_sid = sid
        indices[sid] = []
        targets[sid] = main_target
        sources[sid] = main_source 
    if severity == 'no-error':
        indices[sid].append([-1,-1,'no-error'])
    elif severity == 'minor':
        #find indices in target
        start_indices = find_indices(target,'<v>')
        end_indices = find_indices(target,'</v>')
        if start_indices != None and end_indices != None :
            if len(start_indices)>2 or len(end_indices)>2:
                print("LONGER than anticipated")
            else:
                start = start_indices[0]
                end = end_indices[0]-3
                span = main_target[start:end]
                if span.startswith(' '):
                    start = start+1
                if span.endswith(' '):
                    end = end-1
                
                arr = [start,end,'minor']
                indices[sid].append(arr)
        else:
            arr = [-1,-1,'no-error']
    else:
        start_indices = find_indices(target,'<v>')
        end_indices = find_indices(target,'</v>')
        if start_indices != None and end_indices != None :
            if len(start_indices)>2 or len(end_indices)>2:
                print("LONGER than anticipated")
            else:
                start = start_indices[0]
                end = end_indices[0]-3
                span = main_target[start:end]
                if span.startswith(' '):
                    start = start+1
                if span.endswith(' '):
                    end = end-1
               
                arr = [start,end,'major']
                
                indices[sid].append(arr)
        else:
            arr = [-1,-1,'no-error']
            
lines = []
for s in indices:
    spans = indices[s]
    source = sources[s]
    target = targets[s]
   
    if len(spans)>0:
        starts = []
        ends = []
        sevs = []
        for sp in spans:
            starts.append(str(sp[0]))
            ends.append(str(sp[1]))
            sevs.append(str(sp[2]))
        starts, ends, sevs = zip(*sorted(zip(starts, ends, sevs)))
        start = " ".join(starts)
        end = " ".join(ends)
        sev = " ".join(sevs)
        idx_splits = s.split('_')
        line = [idx_splits[0],idx_splits[1],idx_splits[2],idx_splits[3],source,target,start,end,sev]
        str_line = "\t".join(line)
        #print(str_line)
        lines.append(str_line)
    
    

In [82]:
outfile = '' ## path to output file for task 2
with open(outfile,'w') as wf:
    for line in lines:
        wf.write(line.strip()+'\n')
    